# BERT Post-Training 

By Asmik Nalmpatian and Lisa Wimmer

Last edited on 31.01.21

For our consulting project: Aspect-Based Sentiment Analysis for Twitter Data of German MPs

Methodology based on: https://www.aclweb.org/anthology/N19-1242.pdf

**Prepare GPU:**

1. Check: Edit --> Notebook settings -> Hardware accelerator -> *GPU* 


2. Datasets are uploaded in *content*-folder: *germeval.txt* 

# Google Colab GPU Connection

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# Identify and specify the GPU as the device
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# # Clear RAM for more memory
#!nvidia-smi
#!kill 418.67

Tue Feb  9 13:57:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    25W /  75W |   7603MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#!pip install torch==1.4.0
#!pip install transformers #==3.5.1
#!pip install pytorch_pretrained_bert

import logging
import argparse
import random
import json
import collections

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#from transformers import BertTokenizer
#from pytorch_pretrained_bert import tokenization
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel, BertPreTrainingHeads, BertPreTrainedModel 
from pytorch_pretrained_bert.optimization import BertAdam

# Prepare Tweets for Post-Training

Create functions

In [ ]:
class TrainingInstance(object):
    """A single training instance (sentence pair)."""

    def __init__(self, tokens, segment_ids, masked_lm_positions, masked_lm_labels,
                 is_random_next):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.is_random_next = is_random_next
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels

    def __str__(self):
        s = ""
        s += "tokens: %s\n" % (" ".join(
            [x for x in self.tokens]))
        s += "segment_ids: %s\n" % (" ".join([str(x)
                                              for x in self.segment_ids]))
        s += "is_random_next: %s\n" % self.is_random_next
        s += "masked_lm_positions: %s\n" % (" ".join(
            [str(x) for x in self.masked_lm_positions]))
        s += "masked_lm_labels: %s\n" % (" ".join(
            [x for x in self.masked_lm_labels]))
        s += "\n"
        return s

    def __repr__(self):
        return self.__str__()

In [ ]:
def write_instance_to_example_files(instances, tokenizer, max_seq_length,
                                    max_predictions_per_seq, output_files):
    """Create np example files from `TrainingInstance`s."""
    
    input_ids_np= np.zeros((len(instances), max_seq_length), np.int16)
    segment_ids_np = np.zeros((len(instances), max_seq_length), np.int16)
    input_mask_np = np.zeros((len(instances), max_seq_length), np.int16)
    masked_lm_ids_np = -np.ones((len(instances), max_seq_length), np.int16)
    masked_lm_weights_np = np.zeros((len(instances), max_seq_length), np.int16)
    next_sentence_labels_np = np.zeros((len(instances), ), np.int16)
    
    for (inst_index, instance) in enumerate(instances):
        input_ids = tokenizer.convert_tokens_to_ids(instance.tokens)
        segment_ids = list(instance.segment_ids)
        input_mask = [1] * len(input_ids)

        assert len(input_ids) <= max_seq_length

        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            segment_ids.append(0)
            input_mask.append(0)

        assert len(input_ids) == max_seq_length
        assert len(segment_ids) == max_seq_length
        assert len(input_mask) == max_seq_length

        masked_lm_ids = [-1]*len(instance.tokens)
        for ix, ids in enumerate(tokenizer.convert_tokens_to_ids(instance.masked_lm_labels) ):
            masked_lm_ids[instance.masked_lm_positions[ix]]=ids
        masked_lm_weights = [1.0] * len(masked_lm_ids)

        while len(masked_lm_ids) < max_seq_length:
            masked_lm_ids.append(-1) #ignore index for pytorch
            masked_lm_weights.append(0.0)

        next_sentence_label = 1 if instance.is_random_next else 0
            
        input_ids_np[inst_index]=input_ids 
        segment_ids_np[inst_index] = segment_ids
        input_mask_np[inst_index] = input_mask
        masked_lm_ids_np[inst_index] = masked_lm_ids
        masked_lm_weights_np[inst_index] = masked_lm_weights
        next_sentence_labels_np[inst_index] = next_sentence_label

        if inst_index < 5:
            logging.info("*** Example ***")
            logging.info("tokens: %s" % " ".join([x for x in instance.tokens]))
        
    np.savez_compressed(output_files, 
                        input_ids=input_ids_np, 
                        input_mask = input_mask_np, 
                        segment_ids = segment_ids_np, 
                        masked_lm_ids = masked_lm_ids_np, 
                        masked_lm_weights = masked_lm_weights_np, 
                        next_sentence_labels = next_sentence_labels_np)

In [ ]:
def create_training_instances(input_files, tokenizer, max_seq_length,
                              dupe_factor, short_seq_prob, masked_lm_prob,
                              max_predictions_per_seq, rng):
    """Create `TrainingInstance`s from raw text."""
    all_documents = [[]]

    # Input file format:
    # (1) One sentence per line. These should ideally be actual sentences, not
    # entire paragraphs or arbitrary spans of text. (Because we use the
    # sentence boundaries for the "next sentence prediction" task).
    # (2) Blank lines between documents. Document boundaries are needed so
    # that the "next sentence prediction" task doesn't span between documents.

    with open(input_files, "r") as reader:
        while True:
            line = reader.readline()
            if not line:
                break
            line = line.strip()

            # Empty lines are used as document delimiters
            if not line:
                all_documents.append([])
            tokens = tokenizer.tokenize(line)
            if tokens:
                all_documents[-1].append(tokens)

    # Remove empty documents
    all_documents = [x for x in all_documents if x]
    rng.shuffle(all_documents)

    vocab_words = list(tokenizer.vocab.keys())
    instances = []
    for _ in range(dupe_factor):
        for document_index in range(len(all_documents)):
            instances.extend(
                create_instances_from_document(
                    all_documents, document_index, max_seq_length, short_seq_prob,
                    masked_lm_prob, max_predictions_per_seq, vocab_words, rng))

    rng.shuffle(instances)
    return instances

In [ ]:
def create_instances_from_document(
        all_documents, document_index, max_seq_length, short_seq_prob,
        masked_lm_prob, max_predictions_per_seq, vocab_words, rng):
    """Creates `TrainingInstance`s for a single document."""
    document = all_documents[document_index]

    # Account for [CLS], [SEP], [SEP]
    max_num_tokens = max_seq_length - 3

    # We *usually* want to fill up the entire sequence since we are padding
    # to `max_seq_length` anyways, so short sequences are generally wasted
    # computation. However, we *sometimes*
    # (i.e., short_seq_prob == 0.1 == 10% of the time) want to use shorter
    # sequences to minimize the mismatch between pre-training and fine-tuning.
    # The `target_seq_length` is just a rough target however, whereas
    # `max_seq_length` is a hard limit.
    target_seq_length = max_num_tokens
    if rng.random() < short_seq_prob:
        target_seq_length = rng.randint(2, max_num_tokens)

    # We DON'T just concatenate all of the tokens from a document into a long
    # sequence and choose an arbitrary split point because this would make the
    # next sentence prediction task too easy. Instead, we split the input into
    # segments "A" and "B" based on the actual "sentences" provided by the user
    # input.
    instances = []
    current_chunk = []
    current_length = 0
    i = 0
    while i < len(document):
        segment = document[i]
        current_chunk.append(segment)
        current_length += len(segment)
        if i == len(document) - 1 or current_length >= target_seq_length:
            if current_chunk:
                # `a_end` is how many segments from `current_chunk` go into the `A`
                # (first) sentence.
                a_end = 1
                if len(current_chunk) >= 2:
                    a_end = rng.randint(1, len(current_chunk) - 1)

                tokens_a = []
                for j in range(a_end):
                    tokens_a.extend(current_chunk[j])

                tokens_b = []
                # Random next
                is_random_next = False
                if len(current_chunk) == 1 or rng.random() < 0.5:
                    is_random_next = True
                    target_b_length = target_seq_length - len(tokens_a)

                    # This should rarely go for more than one iteration for large
                    # corpora. However, just to be careful, we try to make sure that
                    # the random document is not the same as the document
                    # we're processing.
                    for _ in range(10):
                        random_document_index = rng.randint(
                            0, len(all_documents) - 1)
                        if random_document_index != document_index:
                            break

                    random_document = all_documents[random_document_index]
                    random_start = rng.randint(0, len(random_document) - 1)
                    for j in range(random_start, len(random_document)):
                        tokens_b.extend(random_document[j])
                        if len(tokens_b) >= target_b_length:
                            break
                    # We didn't actually use these segments so we "put them back" so
                    # they don't go to waste.
                    num_unused_segments = len(current_chunk) - a_end
                    i -= num_unused_segments
                # Actual next
                else:
                    is_random_next = False
                    for j in range(a_end, len(current_chunk)):
                        tokens_b.extend(current_chunk[j])
                truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng)

                assert len(tokens_a) >= 1
                assert len(tokens_b) >= 1

                tokens = []
                segment_ids = []
                tokens.append("[CLS]")
                segment_ids.append(0)
                for token in tokens_a:
                    tokens.append(token)
                    segment_ids.append(0)

                tokens.append("[SEP]")
                segment_ids.append(0)

                for token in tokens_b:
                    tokens.append(token)
                    segment_ids.append(1)
                tokens.append("[SEP]")
                segment_ids.append(1)

                (tokens, masked_lm_positions,
                 masked_lm_labels) = create_masked_lm_predictions(
                     tokens, masked_lm_prob, max_predictions_per_seq, vocab_words, rng)
                instance = TrainingInstance(
                    tokens=tokens,
                    segment_ids=segment_ids,
                    is_random_next=is_random_next,
                    masked_lm_positions=masked_lm_positions,
                    masked_lm_labels=masked_lm_labels)
                instances.append(instance)
            current_chunk = []
            current_length = 0
        i += 1

    return instances


MaskedLmInstance = collections.namedtuple("MaskedLmInstance",
                                          ["index", "label"])

In [ ]:
def create_masked_lm_predictions(tokens, masked_lm_prob,
                                 max_predictions_per_seq, vocab_words, rng):
    """Creates the predictions for the masked LM objective."""

    cand_indexes = []
    for (i, token) in enumerate(tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue
        cand_indexes.append(i)

    rng.shuffle(cand_indexes)

    output_tokens = list(tokens)

    num_to_predict = min(max_predictions_per_seq,
                         max(1, int(round(len(tokens) * masked_lm_prob))))

    masked_lms = []
    covered_indexes = set()
    for index in cand_indexes:
        if len(masked_lms) >= num_to_predict:
            break
        if index in covered_indexes:
            continue
        covered_indexes.add(index)

        masked_token = None
        # 80% of the time, replace with [MASK]
        if rng.random() < 0.8:
            masked_token = "[MASK]"
        else:
            # 10% of the time, keep original
            if rng.random() < 0.5:
                masked_token = tokens[index]
            # 10% of the time, replace with random word
            else:
                masked_token = vocab_words[rng.randint(
                    0, len(vocab_words) - 1)]

        output_tokens[index] = masked_token

        masked_lms.append(MaskedLmInstance(index=index, label=tokens[index]))

    masked_lms = sorted(masked_lms, key=lambda x: x.index)

    masked_lm_positions = []
    masked_lm_labels = []
    for p in masked_lms:
        masked_lm_positions.append(p.index)
        masked_lm_labels.append(p.label)

    return (output_tokens, masked_lm_positions, masked_lm_labels)

In [ ]:
def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens, rng):
    """Truncates a pair of sequences to a maximum sequence length."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_num_tokens:
            break

        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if rng.random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()

Apply: Create training instances out of Germeval Tweets

In [ ]:

import pandas as pd
df = pd.read_csv('./drive/MyDrive/BERT_Files/BERT_Post-Training/tweets_unlabeled.txt' , delimiter=',', header = 1)


In [ ]:
len(df)

29715

In [ ]:
random.seed(12345)
np.random.seed(12345)
    
tokenizer = BertTokenizer.from_pretrained("./drive/MyDrive/BERT_Files/bert_german_cased/") # "bert-base-german-cased") 
rng = random.Random(12345)
    
instances = create_training_instances(
        input_files = './drive/MyDrive/BERT_Files/BERT_Post-Training/tweets_unlabeled.txt', # germeval.txt
        tokenizer = tokenizer, 
        max_seq_length = 320, 
        dupe_factor = 10, 
        short_seq_prob = 0.1, 
        masked_lm_prob = 0.15, 
        max_predictions_per_seq = 40, 
        rng = rng)

write_instance_to_example_files(
    instances = instances, 
    tokenizer = tokenizer, 
    max_seq_length = 320, 
    max_predictions_per_seq = 40, 
    output_files = "tweets_unlabeled.npz")


# The actual training process

In [ ]:
class BertForMTPostTraining(BertPreTrainedModel):
    def __init__(self, config):
        super(BertForMTPostTraining, self).__init__(config)
        self.bert = BertModel(config)
        self.cls = BertPreTrainingHeads(config, self.bert.embeddings.word_embeddings.weight)
        self.qa_outputs = torch.nn.Linear(config.hidden_size, 2)
        self.apply(self.init_bert_weights)

    def forward(self, mode, input_ids, token_type_ids=None, attention_mask=None, masked_lm_labels=None, next_sentence_label=None, start_positions=None, end_positions=None):
        
        sequence_output, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        
        if mode=="review":
            prediction_scores, seq_relationship_score = self.cls(sequence_output, pooled_output)
            if masked_lm_labels is not None and next_sentence_label is not None:
                loss_fct = torch.nn.CrossEntropyLoss(ignore_index=-1)
                masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), masked_lm_labels.view(-1))        
                next_sentence_loss = loss_fct(seq_relationship_score.view(-1, 2), next_sentence_label.view(-1))
                total_loss = masked_lm_loss + next_sentence_loss
                
                return total_loss
            else:
                return prediction_scores, seq_relationship_score

        elif mode=="squad":
            logits = self.qa_outputs(sequence_output)
            start_logits, end_logits = logits.split(1, dim=-1)
            start_logits = start_logits.squeeze(-1)
            end_logits = end_logits.squeeze(-1)

            if start_positions is not None and end_positions is not None:
                # If we are on multi-GPU, split add a dimension
                if len(start_positions.size()) > 1:
                    start_positions = start_positions.squeeze(-1)
                if len(end_positions.size()) > 1:
                    end_positions = end_positions.squeeze(-1)
                # sometimes the start/end positions are outside our model inputs, we ignore these terms
                ignored_index = start_logits.size(1)
                start_positions.clamp_(0, ignored_index)
                end_positions.clamp_(0, ignored_index)

                loss_fct = torch.nn.CrossEntropyLoss(ignore_index=ignored_index)
                start_loss = loss_fct(start_logits, start_positions)
                end_loss = loss_fct(end_logits, end_positions)
                qa_loss = (start_loss + end_loss) / 2
                return qa_loss
            else:
                return start_logits, end_logits
        else:
            raise Exception("unknown mode.")


logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def warmup_linear(x, warmup=0.002):
    if x < warmup:
        return x/warmup
    return 1.0 - x

In [ ]:
# Training arguments

gradient_accumulation_steps = 2
train_batch_size = int(16 / gradient_accumulation_steps)   
review_data_dir = ''
num_train_steps = 12
fp16 = False
learning_rate = 3e-5
warmup_proportion = 0.1
loss_scale = 2
save_checkpoints_steps = 10000
output_dir = ''
random.seed(12345)
np.random.seed(12345)
torch.manual_seed(12345)


In [ ]:
review_train_examples = np.load("tweets_unlabeled.npz")
    
# load bert pre-train data.
review_train_data = TensorDataset(
    torch.from_numpy(review_train_examples["input_ids"]),
    torch.from_numpy(review_train_examples["segment_ids"]),
    torch.from_numpy(review_train_examples["input_mask"]),        
    torch.from_numpy(review_train_examples["masked_lm_ids"]),
    torch.from_numpy(review_train_examples["next_sentence_labels"]) )
    
review_train_dataloader = DataLoader(review_train_data, 
                                     sampler=RandomSampler(review_train_data), 
                                     batch_size = train_batch_size , 
                                     drop_last=True)

# we do not have any validation for pretuning
model = BertForMTPostTraining.from_pretrained("./drive/MyDrive/BERT_Files/bert_german_cased/") # "bert-base-german-cased"

model.cuda()
        
# Prepare optimizer
param_optimizer = [(k, v) for k, v in model.named_parameters() if v.requires_grad==True]
param_optimizer = [n for n in param_optimizer if 'pooler' not in n[0]]
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
t_total = num_train_steps
        
optimizer = BertAdam(optimizer_grouped_parameters,
                             lr = learning_rate,
                             warmup = warmup_proportion,
                             t_total = t_total)

In [ ]:
global_step=0
step=0
batch_loss=0.
model.train()
model.zero_grad()
    
training=True

review_iter=iter(review_train_dataloader)
    
while training:
    try:
        batch = next(review_iter)
    except:
        review_iter=iter(review_train_dataloader)
        batch = next(review_iter)
        
    batch = tuple(t.cuda() for t in batch)
    
    input_ids, segment_ids, input_mask, masked_lm_ids, next_sentence_labels = batch
    
    review_loss = model("review", input_ids.long(), segment_ids.long(), input_mask.long(), masked_lm_ids.long(), next_sentence_labels.long(), None, None)


    batch = tuple(t.cuda() for t in batch)
    input_ids, segment_ids, input_mask, start_positions, end_positions = batch

    loss=review_loss 

    if gradient_accumulation_steps > 1:
        loss = loss / gradient_accumulation_steps
    batch_loss+=loss
    if fp16:
        optimizer.backward(loss)
    else:
        loss.backward()
    
    if (step + 1) % gradient_accumulation_steps == 0:
        # modify learning rate with special warm up BERT uses
        lr_this_step = learning_rate * warmup_linear(global_step/t_total, warmup_proportion)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr_this_step
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1
        if global_step % 50 ==0:
            logging.info("step %d batch_loss %f ", global_step, batch_loss)
        batch_loss=0.

        if global_step % save_checkpoints_steps==0:
            model.float()
            torch.save(model.state_dict(), os.path.join(output_dir, "pytorch_model_"+str(global_step)+".bin") )
            if args.fp16:
                model.half()
        if global_step>=num_train_steps:
            training=False
            break
    step+=1
model.float()
torch.save(model.state_dict(), "pytorch_model.bin" )

/usr/local/lib/python3.7/dist-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
